In [ ]:
from primaite.simulator.network.networks import arcd_uc2_network
%load_ext autoreload
%autoreload 2

In [ ]:
net = arcd_uc2_network()

In [ ]:
random_node = list(net.nodes.keys())[0]
f = net.nodes[random_node].file_system.create_file(file_name="testfile")

In [ ]:
f.describe_state()

In [ ]:
def test_file_observation():
    from primaite.simulator.sim_container import Simulation
    from primaite.simulator.network.hardware.nodes.computer import Computer
    from primaite.game.actor.observations import FileObservation

    sim = Simulation()
    pc = Computer(hostname="beep", ip_address="123.123.123.123", subnet_mask="255.255.255.0")
    sim.network.add_node(pc)
    f = pc.file_system.create_file(file_name="dog.png")

    dog_file_obs = FileObservation(where=['network','nodes',pc.uuid,'file_system'])
    print(sim.describe_state())
test_file_observation()

In [1]:
from primaite.simulator.sim_container import Simulation
from primaite.simulator.network.hardware.nodes.computer import Computer
from primaite.game.actor.observations import FileObservation

sim = Simulation()
pc = Computer(hostname="beep", ip_address="123.123.123.123", subnet_mask="255.255.255.0")
sim.network.add_node(pc)
f = pc.file_system.create_file(file_name="dog.png")

state = sim.describe_state()

dog_file_obs = FileObservation(where=['network','nodes',pc.uuid,'file_system', 'folders','root','files','dog.png'])
o = dog_file_obs(state)
o

2023-09-21 10:41:35,339: Added node f03fec1b-927d-4d5a-8de9-1ef426052932 to Network f7400348-31e5-440e-8eb5-42366326d9d1


{'health_status': 1}

In [4]:
dog_file_obs.space

Dict(health_status:Discrete(6))